In [ ]:
import locale
locale.setlocale(locale.LC_ALL, "de_AT.UTF-8");

In [ ]:
import sys, os.path
sys.path.append(os.path.abspath("../src"))

In [ ]:
%matplotlib inline
%precision %.4f
#%load_ext snakeviz
import colorcet
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.dates
import matplotlib.ticker
import matplotlib.patches
import matplotlib.colors
import seaborn as sns
from importlib import reload
from itertools import count
from datetime import date, datetime, timedelta
from IPython.display import display, Markdown, HTML
import textwrap
from functools import partial
from cycler import cycler
import re
import util

import cov
cov=reload(cov)

pd.options.display.precision = 4

cov = reload(cov)

plt.rcParams['figure.figsize'] = (16*0.7,9*0.7)
plt.rcParams['figure.dpi'] =  120 #80
plt.rcParams['figure.facecolor'] = '#fff'
sns.set_theme(style="whitegrid")
sns.set_palette('tab10')

plt.rcParams['image.cmap'] = cov.un_cmap

pd.options.display.max_rows = 120
pd.options.display.min_rows = 40

In [ ]:
util.DATAROOT

In [ ]:
from pathlib import Path

def _ks_colnames(sfx):
    return [c + sfx for c in KS_BASECOLS]
KS_BASECOLS = ["insured", "cases", "cases_p1000", "active_end", "active_end_p1000"]
KS_COLNAMES = ["id", "insurer",] + _ks_colnames("-ArbeiterInnen") + _ks_colnames("-Angestellte")
KS_FNAME_RE = re.compile("Mb_(\d\d)(\d\d)")

def load_ks(pth):
    m = KS_FNAME_RE.fullmatch(pth.stem)
    if not m:
        raise ValueError("Unexpected filename stem: '" + pth.stem + "' in: " + str(pth))
    year = 2000 + int(m.group(1))
    month = int(m.group(2))
    
    ks = pd.read_excel(
        pth,
        sheet_name="Tab16",
        skiprows=9,
        header=None,
        names=KS_COLNAMES
        )
    ks.dropna(thresh=3, inplace=True)
    if pd.isna(ks["insurer"].iloc[0]):
        raise ValueError("Missing insurer")
    ks["date"] = (ks["id"] % 2).map(
        lambda odd: pd.Period(year=year if odd else year - 1, month=month, freq="M"))
    return ks

def collect_ks():
    ks = pd.concat([load_ks(pth) for pth in (util.DATAROOT / "sozialversicherung-monatsberichte").glob("Mb_????.xls*")])
    ks["insurer"].replace(
        "I n s g e s a m t|insgesamt|ASVG-Krankenkassen", "Insgesamt", regex=True, inplace=True)
    ks["insurer"].replace(
        r"VA f\. Eisenb\.u\.Bergbau|Abteilung A",
        "VA f. Eisenb.u.Bergbau Abteilung A", regex=True, inplace=True)
    #ks["insurer"].replace("Gebietskrankenkassen", "Österr. Gesundheitskasse", inplace=True)
    
    # Comment next line out to represent the time series break
    ks["insurer"] = ks["insurer"].str.removeprefix("GKK ")
    ks["insurer"] = ks["insurer"].str.strip()
    ks.ffill(inplace=True, limit=1)
    #display(ks)
    ks = pd.wide_to_long(
        ks, KS_BASECOLS, ["id", "date", "insurer"], "employment", sep="-",
        suffix="(ArbeiterInnen|Angestellte)").reset_index()
    mask = np.round(ks["cases"] / ks["insured"] * 1000) != ks["cases_p1000"]
    if mask.any():
        raise ValueError("Mismatch of cases_p1000 at " + ks.loc[mask].to_csv(sep=";"))
    mask = np.round(ks["active_end"] / ks["insured"] * 1000) != ks["active_end_p1000"]
    if mask.any():
        raise ValueError("Mismatch of active_end_p1000 at " + ks.loc[mask].to_csv(sep=";"))
    ks.drop(columns=["id", "cases_p1000", "active_end_p1000"], inplace=True)
    ks.drop_duplicates(
        # Comment the next line to see
        # (a) corrections in 2020 08-10 totals,
        # (b) some changes through ÖGKK -> ÖGK
        subset=["date", "insurer", "employment"], keep="last",
        inplace=True)
    ks.set_index(["date", "insurer", "employment"], inplace=True, verify_integrity=True)
    return ks.sort_index()
    
ks = collect_ks()
ks_o = ks.copy()

In [ ]:
ks=ks_o.copy()

ks["jun_year"] = ks.index.get_level_values("date").asfreq("A-JUN").year - 1
ks["jun_year_l"] = ks["jun_year"].astype(str) + "/" + ((ks["jun_year"] + 1) % 100).astype(str) 
ks["jun_month"] = (ks.index.get_level_values("date").month - 1 + (12 - 6)) % 12 + 1
ks["cases_n"] = ks["cases"] / ks.index.get_level_values("date").days_in_month * 30
ks["m_start"] = ks.index.get_level_values("date").start_time

#ks["qyear"] = ks["qdate"].dt.qyear
ks = cov.add_sum_rows(ks, "employment", "Insgesamt", {
    "insured": "sum", "cases": "sum", "cases_n": "sum", "active_end": "sum",
    "jun_year": "first", "jun_year_l": "first", "jun_month": "first", "m_start": "first"})

ks["active_end_pp"] = ks["active_end"] / ks["insured"]
ks["cases_pp_n30"] = ks["cases_n"] / ks["insured"]
ks["cases_pp_raw"] = ks["cases"] / ks["insured"]
#ks_s["cases_pp_n30"] = ks_s["cases_n"] / ks_s["insured"]
#ks_s["active_end_pp"] = ks_s["active_end"] / ks_s["insured"]
#ks["m_start"] = ks0["date"].dt.start_time
#display(ks_s.query("insurer == 'Insgesamt' and employment != 'angest'")[["jun_year", "month"]])

In [ ]:
def plt_ks_yoy(y, bl, empl="Insgesamt"):
    whatlabel = (
        "Krankenstandfälle/Monat (30-Tage-normiert)" if y == "cases_pp_n30" else
        "Laufende Krankenstände am Monatsende" if y == "active_end_pp" else None)
    empllabel = "" if empl == "Insgesamt" else f" (nur {empl})"
    fig, ax = plt.subplots()
    #display(ks.dtypes)
    #ax.set_xlabels(["])
    blq = "Insgesamt" if bl == 'Österreich' else bl
    pdata = ks.query(f"insurer == '{blq}' and employment == '{empl}'").sort_index().reset_index()
    pal = sns.color_palette("tab10", n_colors=pdata["jun_year_l"].nunique())[::-1]
    pltargs = dict(x="jun_month", y=y, hue="jun_year_l", palette=pal, sort=False, markersize=9)
    sns.lineplot(pdata,
                  #lw=3,
                #size=pdata["jun_year_l"].map(lambda y: 3 if y == pdata.iloc[0]["jun_year_l"] else 2),
                #sizes=(2, 4),
                marker="o" if y != "active_end_pp" else None,
                lw=3, **pltargs)
    if y == "active_end_pp":
        wday = pdata["date"].dt.end_time.dt.dayofweek
        is_weekend = wday > 4
        is_monday = wday == 0
        pd0 = pdata.copy()
        pd0.loc[is_monday | is_weekend, y] = np.nan
        sns.lineplot(
            pd0,
            marker="o",
            lw=0, **pltargs, legend=False)
        pd0 = pdata.copy()
        pd0.loc[~is_monday, y] = np.nan
        sns.lineplot(
            pd0,
            marker="X",
            lw=0, **pltargs, legend=False, mew=0)
    
    cov.set_percent_opts(ax)
    ax.axvline(12 - 6 + 1 - 0.5, color="k", ls="--", lw=1, zorder=1)
    ax.set_xticks(np.arange(1, 12+1))
    ax.set_xticklabels(["Jul", "Aug", "Sep", "Okt", "Nov", "Dez", "Jän", "Feb", "Mär", "Apr", "Mai", "Jun"])
    ax.set_ylim(bottom=0)
    ax.set_xlabel("Monat")
    ax.legend(title="Jahr bzw. Saison")
    #ax.set_ylabel("Anteil der Versicherten (30-Tage-normiert)")
    ax.set_ylabel("Fälle/Versicherte")
    #fig.suptitle("Aktive Krankenstandfälle am Monatsende als Anteil der Versicherten", y=0.93)
    fig.suptitle(f"{bl}{empllabel}: {whatlabel} als Anteil der Versicherten", y=0.93)
    kwargs = dict(ax=ax, mms=pdata, ycol=y, cats="jun_year", x="jun_month", colorize=pal)
    cov.labelend2(**kwargs, shorten=lambda x: str((x + 1) % 100))
    cov.labelend2(**kwargs, ends=(True, False), shorten=lambda x: "17" if x == 2016 else str(x % 100))
    ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(0.02))
    cov.stampit(fig, "Monatsberichte des österr. Dachverband der Sozialversicherungsträger")
    ax.legend(ncol=3, title="Jahr bzw. Saison")
plt_ks_yoy("active_end_pp", "Österreich")
if False:
    for bl in cov.BUNDESLAND_BY_ID.values():
        plt_ks_yoy("cases_pp_n30", bl)
plt_ks_yoy("cases_pp_n30", "Österreich", "Insgesamt")
plt_ks_yoy("cases_pp_n30", "Österreich", "ArbeiterInnen")
plt_ks_yoy("cases_pp_n30", "Österreich", "Angestellte")

In [ ]:
fig, ax = plt.subplots()
ks0 = ks.query("employment == 'Insgesamt'").reset_index()#.query("employment == 'angest'").reset_index()
sns.lineplot(ks0.query(
    "insurer in ('Vorarlberg', 'Tirol', 'Salzburg', 'Oberösterreich', 'Kärnten', 'Steiermark', 'Niederösterreich', 'Wien', 'Burgenland')"
), hue="insurer",# style="employment",
             x="m_start", y="cases_pp_n30", marker=".", mew=0)
cov.set_percent_opts(ax, decimals=1)
ax.xaxis.set_major_locator(matplotlib.dates.YearLocator())
ax.xaxis.set_minor_formatter(matplotlib.dates.DateFormatter("%b"))
ax.xaxis.set_minor_locator(matplotlib.dates.MonthLocator([1, 4, 7, 10]))
ax.set_ylim(bottom=0)

ax.grid(which="minor", lw=0.3)
#fig.autofmt_xdate()
ax.tick_params(axis="x", which="major", pad=11)
#cov.set_logscale(ax)
ax.legend(ncol=2, loc="lower left")
fig.suptitle("Krankenstandzugang/Monat (normiert auf 30 Tage) in % der Versicherten")
ax.set_title("Arbeiter + Angestellte, je Versichernder ÖGK")
ax.set_xlabel("Berichtsmonat")
ax.set_ylabel("Fälle/Versicherte (30-Tage-normiert)")

In [ ]:
logscale = False
fig, ax = plt.subplots()
if logscale:
    cov.set_logscale(ax)
fig.suptitle("Krankenstandzugang/Monat (normiert auf 30 Tage) in % der Versicherten")
ax.set_title("Über alle im Bericht erfassten Träger, je Arbeitsverhältnis")

pdata = ks.query("insurer == 'Insgesamt'").reset_index()
pargs = dict(x="m_start", y="cases_pp_n30", estimator=None, marker=".", mew=0, ax=ax)
ax = sns.lineplot(pdata.query("employment != 'Insgesamt'"), hue="employment", ls="--", **pargs)
sns.lineplot(pdata.query("employment == 'Insgesamt'"),
             lw=2, markersize=10, color="grey", label="Insgesamt", **pargs)
cov.set_percent_opts(ax, decimals=1)
ax.xaxis.set_major_locator(matplotlib.dates.YearLocator())
ax.xaxis.set_minor_formatter(matplotlib.dates.DateFormatter("%b"))
ax.xaxis.set_minor_locator(matplotlib.dates.MonthLocator([1, 4, 7, 10]))
ax.grid(which="minor", lw=0.3)
#fig.autofmt_xdate()
ax.tick_params(axis="x", which="major", pad=11)
if not logscale:
    ax.set_ylim(bottom=0)

ax.set_xlabel("Berichtsmonat")
ax.set_ylabel("Fälle/Versicherte (30-Tage-normiert)")

ax.legend()

In [ ]:
ks.query("insurer == 'Österr. Gesundheitskasse' and employment == 'all' and date.dt.year == 2022")["cases_pp_n30"].sum()

In [ ]:
if False:
    # Correct for bad case removal https://twitter.com/zeitferne/status/1584103659477925888
    def stitchcsv(oldpath, newpath):
        oldbegdate = pd.to_datetime("16.03.2020 00:00:00", dayfirst=True)
        oldenddate = pd.to_datetime("31.03.2020 00:00:00", dayfirst=True)
        olddf = pd.read_csv(oldpath, sep=";", encoding="utf-8")
        newdf = pd.read_csv(newpath, sep=";", encoding="utf-8")
        dcol = "MeldeDatum" if "MeldeDatum" in olddf.columns else "Time" if "Time" in olddf.columns else "Datum"
        if not dcol:
            raise ValueError("Could not find datecol in " + str(oldpath))

        oldt = pd.to_datetime(olddf[dcol], dayfirst=True)
        newt = pd.to_datetime(newdf[dcol], dayfirst=True)

        def addcol(cols, cname):
            if cname in olddf.columns:
                cols.append(cname)

        catcols = []
        addcol(catcols, "BundeslandID")
        addcol(catcols, "AltersgruppeID")
        addcol(catcols, "Geschlecht")
        addcol(catcols, "GKZ")

        scols = [c for c in olddf.columns if c.endswith("Sum")]
        addcol(scols, "Anzahl")
        addcol(scols, "AnzahlTot")
        addcol(scols, "AnzahlGeheilt")

        oldpart = olddf[(oldt >= oldbegdate) & (oldt < oldenddate)]
        def tform(chunk):
            for scol in scols:
                oldbase = chunk[scol]


    bpath = Path(r"T:\Users\Christian\myProgs\covidat\coronaDAT_patch\final")
    for fname in (
        "CovidFaelle_Altersgruppe.csv",
        "CovidFaelle_Timeline.csv",
        "CovidFaelle_Timeline_GKZ.csv",
        "CovidFaelleDelta.csv"):

        stitched = stitchcsv(
            bpath / "20221021_140201_orig_csv_ages" / fname,
            bpath / "20230630_140201_orig_csv_ages" / fname)
        if stitched is not None:
            stitched.to_csv(bpath / fname, encoding="utf-8", sep=";")

In [ ]:
medshort = pd.read_csv(
    util.COLLECTROOT / "medshort/medshort.csv",
    sep=";")
medshort["Datum"] = cov.parseutc_at(medshort["Datum"], format=cov.ISO_DATE_FMT)
medshort.set_index("Datum", inplace=True)
if False:
    fig, ax = plt.subplots()
    ax.plot(medshort["NLimitedMeds"], marker="o")
    ax.set_ylim(bottom=0, top=medshort["NLimitedMeds"].max() * 1.05)
    cov.set_date_opts(ax)
    fig.suptitle("Anzahl Einträge in Liste der Meldungen zu Vertriebseinschränkungen von Arzneispezialitäten")
    ax.set_title("Quelle: medicineshortage.basg.gv.at/vertriebseinschraenkungen")

In [ ]:
def labelavail(avail):
    return (avail
        .replace("verfügbar gemäß §4 (1)", "§4(1) (~Mangel entgegen Herstellerangabe)")
        .replace("teilweise verfügbar", "gewisse Größen voll verfügbar (Verwechslung mit eing. verf. möglich)"))

In [ ]:
med_x = pd.read_csv(util.COLLECTROOT / "medshort/medshort_ex.csv", encoding="utf-8", sep=";"
                   ).query("Datum >= '2022-12-01' and (SourceFormat == 'xml' or Datum <= '2023-06-25')")
display(med_x.query("Availability == 'verfügbar'"))
med_x = med_x.query("Availability != 'verfügbar'")
med_x["Datum"] = pd.to_datetime(med_x["Datum"], format="%Y-%m-%d")
med_x["Datum"].freq = "D"
#med_x["Datum_n"] = matplotlib.dates.date2num(med_x["Datum"])
AvailC = pd.CategoricalDtype([
    "nicht verfügbar",
    "eingeschränkt verfügbar",
    "verfügbar gemäß §4 (1)",
    "teilweise verfügbar"])
med_x["Avail_c"] = med_x["Availability"].astype(AvailC)
med_x.sort_values(["Datum", "Avail_c"], kind="stable", inplace=True)
#display(med_x)
med_xh = med_x.query("Usage == 'Human'").reset_index()
#ax = sns.area(med_xh, x="Datum", y="N", hue="Availability", marker="o")
#ax = med_xh.pivot(index="Datum", columns="Availability", values="N").resample("D").plot.bar(stacked=True)
#cov.set_date_opts(ax)
#ax.set_ylim(bottom=0)
fig, ax = plt.subplots()
ax.set_axisbelow(False)
#ax.plot(medshort["NLimitedMeds"], marker="o", color="grey",
#        label="Anzahl Einträge insgesamt, inkl. Veterinär: " + str(medshort["NLimitedMeds"].iloc[-1]), zorder=1)
pltr = cov.StackBarPlotter(
    ax, pd.date_range(start=med_xh["Datum"].min(), end=med_xh["Datum"].max(), freq="1D"),
    lw=0, width=1)
ax.set_prop_cycle(color=sns.color_palette("plasma", n_colors=med_x["Avail_c"].nunique()))
for avail, avail_d in med_xh.groupby("Availability", sort=False):
    #display(avail_d)
    pltr(avail_d.set_index("Datum")["N"].resample("1D").interpolate(limit=1).to_numpy(),
         label=labelavail(avail) + ": " + str(avail_d["N"].iloc[-1]))
    
def decorate_medshort(ax, titley, supy, extralabel=""):
    ax.figure.suptitle("Medikamentenmangel in Österreich", y=supy)
    cov.set_date_opts(ax, showyear=True)
    ax.axvline(pd.to_datetime("2023-06-23T12:00:00"), color="k", ls=":",
           label="Umstellung eigener Datensammlung von xlsx auf xml")
    ax.set_title(
    "Darstellung: @zeitferne | Daten: medicineshortage.basg.gv.at/vertriebseinschraenkungen, bis "
    + med_xh.iloc[-1]["Datum"].strftime("%d.%m.%Y")
   # + "\nVerschiedene Packungsgrößen (nicht Dosisstärken) möglichst als gleiches Medikament gezählt"
    + extralabel,
    y=titley)
    ax.set_ylabel("Anzahl Human-Medikamente")
#ax.figure.autofmt_xdate()
decorate_medshort(ax, 1.05, 1.01, " | 1-tägige Lücken interpoliert")
ax.legend(loc="lower left", title="Anzahl Human-Medikamente je Kategorie")
#med_xh

if False:
    ax.text(
        0.5, 1.01,
        "Falls ein Medikament in allen beschränkten Dosisstärken in allen Packungsgrößen beschränkt ist, "
        "aber eine andere Dosisstärke uneingeschränkt verfügbar wäre,\n"
        "wird stattdessen wegen Limitierungen in der Daten-Analyse fälschlicherweise "
        "die stärkere Beschränkung angegeben.",
        fontsize="x-small",
        ha="center",
        transform=ax.transAxes)
display(cov.filterlatest(med_xh))
display(cov.filterlatest(med_xh)["N"].sum())
display(med_xh[med_xh["Datum"] == med_xh["Datum"].min()])
display(med_xh[med_xh["Datum"] == med_xh["Datum"].min()]["N"].sum())


In [ ]:
pdata = med_xh.copy()
med_xh["Availability"] = labelavail(med_xh["Availability"])
ax = sns.lineplot(med_xh, x="Datum", y="N", hue="Availability")
ax.get_legend().remove()
decorate_medshort(ax, supy=1.065, titley=1.14)
fig = ax.figure
fig.legend(*cov.sortedlabels(ax, med_xh, "N", "Availability", fmtval=lambda v: f"{v:.0f}"),
           frameon=False, ncol=2, loc="upper center", bbox_to_anchor=(0.5, 1))
ax.set_ylim(bottom=0)

In [ ]:
display(med_xh[med_xh["Datum"] == pd.to_datetime("2023-04-25")].sum(numeric_only=True))

In [ ]:
def onlyelem(xs):
    xs = iter(xs)
    v = next(xs)
    try:
        next(xs)
    except StopIteration:
        return v
    raise ValueError("More than one element")
grippe = onlyelem(pd.read_html(
    util.DATAROOT / "ages-epi-misc/grippe_20230714_090503.html",
    match="Influenza A/H3N2 Fall",
    thousands=".",
    decimal=","))

In [ ]:
gfallcol = [c for c in grippe.columns if " Fall" in c]
grippe["Woche"] = grippe["Woche"].astype(str)
grippe.set_index("Woche", inplace=True, verify_integrity=True)
npos = grippe[gfallcol].sum(axis=1)
ntest = (npos / grippe["Positiv-Rate (positive Proben/Untersuchte Proben)"]).round()
ppos = grippe[gfallcol].div(ntest, axis=0)
#display(ntest)
#display(grippe[gfallcol])
ppos.rename(columns={c: '% ' + c.replace(" Fall", "-positiv") for c in ppos.columns}, inplace=True)
color = [f"C{i}" for i in range(len(ppos.columns))]
ax = ppos.plot(color=color)
ppos.where(lambda x: x != 0).plot(ax=ax, lw=0, marker="o", markersize=10, alpha=0.8, color=color, legend=False)
fig = ax.figure
ax.set_ylim(bottom=0)
cov.set_percent_opts(ax)
ax.set_ylabel("Positivrate")
fig.suptitle("Influenza in Österreich, Sentinel-Proben 2022/23 (AGES/DINÖ)", y=0.93)

In [ ]:
ax = grippe.plot(y=gfallcol, marker="o", markersize=10, alpha=0.8)
cov.set_logscale(ax)
ax2 = ax.twinx()
grippe.plot(y=[c for c in grippe.columns if "Positiv-Rate" in c], ax=ax2, color="k", marker="s",
           markersize=4)
ax2.grid(False)
ax2.get_legend().remove()
ax.get_legend().remove()
fig = ax.figure
fig.suptitle("Influenza in Österreich, Sentinel-Proben 2022/23 (AGES/DINÖ)", y=1.05)
fig.legend(loc="upper center", ncol=2, frameon=False, bbox_to_anchor=(0.5, 1))
ax.set_ylabel("Anzahl positiver Proben (log)")
ax2.set_ylabel("Positivrate")
cov.set_percent_opts(ax2)
ax.set_ylim(bottom=0.9)
ax2.set_ylim(bottom=0)


In [ ]:
(ntest).plot(marker="o").set_ylim(0)
plt.gcf().suptitle("Anzahl Sentinel-Proben (positiv+negativ) / KW (geschätzt)", y=0.93)